## preprocessing the data and splitting it into training, validation, and testing sets. 

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.utils import to_categorical


# Define constants
IMAGE_SIZE = (128, 128)  # Resize images to 128x128 pixels
NUM_CLASSES = 2  # 2 classes: 'with_mask' and 'without_mask'
BATCH_SIZE = 32

# Function to load and preprocess images
def load_and_preprocess_data(dataset_dir):
    images = []
    labels = []

    # Load images with masks
    mask_dir = os.path.join(dataset_dir, 'with_mask')
    for image_name in os.listdir(mask_dir):
        image_path = os.path.join(mask_dir, image_name)
        image = load_img(image_path, target_size=IMAGE_SIZE)
        image = img_to_array(image) / 255.0  # Normalize pixel values
        images.append(image)
        labels.append(1)  # Label 1 for images with masks

    # Load images without masks
    without_mask_dir = os.path.join(dataset_dir, 'without_mask')
    for image_name in os.listdir(without_mask_dir):
        image_path = os.path.join(without_mask_dir, image_name)
        image = load_img(image_path, target_size=IMAGE_SIZE)
        image = img_to_array(image) / 255.0  # Normalize pixel values
        images.append(image)
        labels.append(0)  # Label 0 for images without masks

    # Convert lists to numpy arrays
    images = np.array(images)
    labels = to_categorical(labels, num_classes=NUM_CLASSES)  # Convert labels to one-hot encoding

    return images, labels

# Load and preprocess data
dataset_dir = "C:\\Users\\ASUS\\Face Mask Detection\\Dataset"
images, labels = load_and_preprocess_data(dataset_dir)

# Split data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training set:", X_train.shape)
print("Validation set:", X_val.shape)
print("Testing set:", X_test.shape)

C:\Users\ASUS\anaconda3\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Training set: (5437, 128, 128, 3)
Validation set: (605, 128, 128, 3)
Testing set: (1511, 128, 128, 3)


### Creating CNN Model

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
def create_model(input_shape):
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    # Flatten layer
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))  # Output layer

    return model

# Create the model
input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)  # Input shape of images (height, width, channels)
model = create_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

### Model Training

In [3]:

# Train the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
170/170 [==============================] - 116s 665ms/step - loss: 0.4205 - accuracy: 0.8188 - val_loss: 0.2421 - val_accuracy: 0.9008
Epoch 2/10
170/170 [==============================] - 115s 673ms/step - loss: 0.2522 - accuracy: 0.8946 - val_loss: 0.2189 - val_accuracy: 0.9124
Epoch 3/10
170/170 [==============================] - 129s 757ms/step - loss: 0.2003 - accuracy: 0.9231 - val_loss: 0.1840 - val_accuracy: 0.9289
Epoch 4/10
170/170 [==============================] - 143s 842ms/step - loss: 0.1684 - accuracy: 0.9349 - val_loss: 0.1892 - val_accuracy: 0.9256
Epoch 5/10
170/170 [==============================] - 156s 915ms/step - loss: 0.1259 - accuracy: 0.9537 - val_loss: 0.1818 - val_accuracy: 0.9322
Epoch 6/10
170/170 [==============================] - 170s 1000ms/step - loss: 0.0893 - accuracy: 0.9676 - val_loss: 0.2213 - val_accuracy: 0.9339
Epoch 7/10
170/170 [==============================] - 206s 1s/step - loss: 0.0628 - accuracy: 0.9779 - val_loss: 0.2323 - v

In [4]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

48/48 [==============================] - 8s 160ms/step - loss: 0.1814 - accuracy: 0.9484
Test Loss: 0.1814316213130951
Test Accuracy: 0.9483785629272461


In [5]:
model.save("face_mask_detection_model.h5")

### Perform Testing of model on video

In [7]:
import cv2
from keras.models import load_model
import numpy as np

# Load pre-trained model
model = load_model("C:\\Users\\ASUS\\Face Mask Detection\\face_mask_detection_model.h5")

# Load the pre-trained face detector from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect faces in an image
def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Function to predict whether the face is wearing a mask
def predict_mask(image, model):
    # Preprocess the image
    image = cv2.resize(image, (128, 128))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize pixel values

    # Predict using the model
    prediction = model.predict(image)
    return prediction

# Open video capture
cap = cv2.VideoCapture("C:\\Users\\ASUS\\Downloads\\Test_video1.mp4")

# Loop through each frame of the video
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces in the frame
    faces = detect_faces(frame)

    # Loop through each detected face
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        
        # Predict whether the face is wearing a mask
        prediction = predict_mask(face_roi, model)
        mask_label = "Mask" if prediction[0][1] > prediction[0][0] else "No Mask"
        color = (0, 255, 0) if mask_label == "Mask" else (0, 0, 255)
        
        # Draw bounding box around the face and put label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, mask_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    
    # Display the frame
    cv2.imshow('Face Mask Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 28ms/step
